In [1]:
import os

In [2]:
list_file = list(os.walk('./part'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
list_file.sort()
    
len(list_file)

In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame()

df

""


In [5]:
from tqdm import tqdm
from bs4 import BeautifulSoup
from lxml import etree
import json

In [6]:
for file in tqdm(list_file, desc='Progress', ncols=77):
    with open('./part/' + file, 'r', encoding='UTF-8') as f:
        html = f.read()
        
        # = = = = = = = = = = = = = = =
        
        soup = BeautifulSoup(html, 'lxml')
        
        # = = = = = = = = = = = = = = =
        
        html = etree.HTML(str(soup))

        title = html.xpath('//h1[@data-scroll-trigger-for-sticky-product-header]/text()')[0].strip()

        # = = = = = = = = = = = = = = =
        
        list_div = html.xpath('//div[contains(@class, "description-block")]')
        
        # = = = = = = = = = = = = = = =
        
        list_oe = [oe.strip() for oe in html.xpath('//div[@class="oe-number"]/descendant-or-self::*/text()')]

        while '' in list_oe:
            list_oe.remove('')

        list_oe = list(set(list_oe))

        oe = ';'.join(list_oe)

        # = = = = = = = = = = = = = = =
        
        part_code = html.xpath('//div[@id="tabMatchingVehicles"]/@data-product-id')[0].strip()
        
        # = = = = = = = = = = = = = = =
        
        key_value = json.loads(html.xpath('//script[@type="application/ld+json"]/text()')[0])
        
        # = = = = = = = = = = = = = = =
        
        df_temp = pd.DataFrame([{'序号': int(file.replace('.txt', '')),
                                 'Sku': key_value['@graph'][0]['sku'].strip(),
                                 'Title': title,
                                 'Manufacturer': key_value['@graph'][0]['manufacturer'].strip(),
                                 'Mpn': key_value['@graph'][0]['mpn'].strip(),
                                 'Price': key_value['@graph'][0]['offers'][0]['priceCurrency'].strip() + ' ' + key_value['@graph'][0]['offers'][0]['price'].strip(),
                                 'Kit': '',
                                 'Vehicle_1': '',
                                 'Vehicle_2': '',
                                 'OE': oe,
                                 'Pic': '',
                                 'Url': '',
                                 'Src': key_value['@graph'][0]['image'].strip(),
                                 'Part_Code': part_code,
                                 'Make_Url': 'https://www.atp-autoteile.de/de/product/vehicles/' + part_code}])
        
        # = = = = = = = = = = = = = = =
        
        for div in list_div:
            if 'delivery-contents' in div.xpath('./@class')[0]:
                list_content = [content.strip() for content in div.xpath('./div[@class="description-content"]/text()')]

                while '' in list_content:
                    list_content.remove('')

                list_amount = [span.text.strip() for span in div.xpath('./div[@class="description-content"]/span[@class="delivery-content-count"]')]

                df_temp.loc[0, div.xpath('./div[@class="description-title"]/text()')[0].strip()] = '\n'.join([amount + ' ' + content for amount, content in zip(list_amount, list_content)])
            elif 'technical-info' in div.xpath('./@class')[0]:
                pass
            else:
                if len(div.xpath('./@style')) != 0:
                    if div.xpath('./@style')[0].strip() != 'display: none;':
                        print(key_value['@graph'][0]['url'].strip())
                        print(div.xpath('./@class')[0])
        
        # = = = = = = = = = = = = = = =
        
        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =
        
df = df.drop_duplicates(ignore_index=True)
df = df.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
df.to_excel('./part.xlsx', index=False)

df

Progress: 100%|██████████████████████████| 1358/1358 [00:59<00:00, 22.85it/s]


,序号,Sku,Title,Manufacturer,Mpn,Price,Kit,Vehicle_1,Vehicle_2,OE,Pic,Url,Src,Part_Code,Make_Url,Lieferumfang
0,1,40579708,ATEC 40579708 – Satz Bremsbeläge + Bremssättel...,ATEC,40579708,EUR 64.94,,,,34111165557;34111160351;34111165556;3411116143...,,,https://www.atp-autoteile.de/media/product/800...,4316658,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Vorderachse\n1x ...
1,2,34001991,BESTPRICE 34001991 – Satz Bremsbeläge + Bremss...,BESTPRICE,34001991,EUR 62.51,,,,34111165557;34111160351;34111165556;3411116143...,,,https://www.atp-autoteile.de/media/product/800...,988398,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Vorderachse\n1x ...
2,3,40579641,ATEC 40579641 – 8x Bremsbeläge + Warnkontakt V...,ATEC,40579641,EUR 45.95,,,,34116794913;34116794917;34116761284;3411676719...,,,https://www.atp-autoteile.de/media/product/200...,4309328,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Vorderachse\n1x ...
3,4,34002361,BESTPRICE 34002361 – 8x Bremsbeläge Vorder- + ...,BESTPRICE,34002361,EUR 44.17,,,,3Q0698451B;8V0698451A;3Q0698451;5QD698151;3Q06...,,,https://www.atp-autoteile.de/media/product/200...,1020670,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Vorderachse\n1x ...
4,5,40579658,ATEC 40579658 – Bremsbeläge +Warnkontakt vorne...,ATEC,40579658,EUR 43.79,,,,"34216761248;SFP000120;34211164581,;34216761250...",,,https://www.atp-autoteile.de/media/product/200...,4309449,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Vorderachse\n1x ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,1354,40004092,BESTPRICE 40004092 – Bremsbeläge hinten (Satz),BESTPRICE,40004092,EUR 5.89,,,,"75519956;95650072,;95565563;95587471,;95587471...",,,https://www.atp-autoteile.de/media/product/800...,3731297,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Hinterachse\n1x ...
1354,1355,31528616,BESTPRICE 31528616 – Bremsbeläge hinten (Satz),BESTPRICE,31528616,EUR 5.84,,,,"34211150556;34211159265;34211117981;,",,,https://www.atp-autoteile.de/media/product/800...,3726194,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Hinterachse\n1x ...
1355,1356,31528852,BESTPRICE 31528852 – Bremsbeläge vorne (Satz) ...,BESTPRICE,31528852,EUR 5.80,,,,,,,https://www.atp-autoteile.de/media/product/800...,3727772,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Vorderachse inkl...
1356,1357,31478334,BESTPRICE 31478334 – Bremsbeläge hinten (Satz),BESTPRICE,31478334,EUR 5.80,,,,"0446632040;0446632030;,",,,https://www.atp-autoteile.de/media/product/800...,3647398,https://www.atp-autoteile.de/de/product/vehicl...,1x Satz (4 Stück) Bremsbeläge Hinterachse\n1x ...
